In [ ]:
!pip install tensorboardX
!pip install torchnet
!pip install datasets

In [2]:
import os
colab = True

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Tesis/tesis/classificador')
else:
    os.chdir('../classificador')

Mounted at /content/drive


In [3]:
import time
import csv
import os

from tqdm import *

import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchvision.transforms import *
import torchnet

from datasets import *
from transforms import *


In [4]:
dataset_dir = '../data/classifier/test.csv'
device = "cuda" if torch.cuda.is_available() else "cpu"
print("loading model...")
model = torch.load('../classificador/1711206440837-resnext29_8_64_sgd_plateau_bs96_lr1.0e-02_wd1.0e-03-best-loss.pth', map_location=device)
model.float()

use_gpu = torch.cuda.is_available()
print('use_gpu', use_gpu)
if use_gpu:
    torch.backends.cudnn.benchmark = True
    model.cuda()

n_mels = 32

feature_transform = Compose([ToMelSpectrogram(n_mels=n_mels), ToTensor('mel_spectrogram', 'input')])
transform = Compose([LoadAudio(), FixAudioLength(), feature_transform])
test_dataset = SpeechCommandsDataset(dataset_dir, transform, silence_percentage=0)
test_dataloader = DataLoader(test_dataset, batch_size=128, sampler=None)
                            # pin_memory=use_gpu, num_workers=args.dataload_workers_nums)

criterion = torch.nn.CrossEntropyLoss()


loading model...
use_gpu True


In [5]:

def test():
    model.eval()  # Set model to evaluate mode

    #running_loss = 0.0
    #it = 0
    correct = 0
    total = 0
    confusion_matrix = torchnet.meter.ConfusionMeter(len(test_dataset.classes))
    predictions = {}
    probabilities = {}

    pbar = tqdm(test_dataloader, unit="audios", unit_scale=test_dataloader.batch_size)
    for batch in pbar:
        inputs = batch['input']
        inputs = torch.unsqueeze(inputs, 1)
        targets = batch['target']

        n = inputs.size(0)

        with torch.no_grad():
            inputs = Variable(inputs)
        targets = Variable(targets, requires_grad=False)

        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # forward
        outputs = model(inputs)
        #loss = criterion(outputs, targets)
        outputs = torch.nn.functional.softmax(outputs, dim=1)

        # statistics
        #it += 1
        #running_loss += loss.data[0]
        pred = outputs.data.max(1, keepdim=True)[1]
        correct += pred.eq(targets.data.view_as(pred)).sum()
        total += targets.size(0)
        # confusion_matrix.add(pred, targets.data)

        filenames = batch['path']
        for j in range(len(pred)):
            fn = filenames[j]
            predictions[fn] = pred[j][0]
            probabilities[fn] = outputs.data[j].tolist()

    accuracy = correct/total
    #epoch_loss = running_loss / it
    print("accuracy: %f%%" % (100*accuracy))
    # print("confusion matrix:")
    # print(confusion_matrix.value())

    return probabilities, predictions


In [6]:
probabilities, predictions = test()

100%|██████████| 5888/5888 [1:22:58<00:00,  1.18audios/s]

accuracy: 99.931046%
